# Sugestão = Criar um classificador para o review dos clientes ao serviço prestado.

    Notas 1, 2 e 3 → 0 (Ruim/Neutro)
    Notas 4 e 5 → 1 (Bom)

processando as tabelas que tem relação com o tempo do serviço

caracteristicas que podem ser boas tiradas dessa tabela:

    1. Tempo de Entrega Real = order_delivered_customer_date - order_purchase_timestamp
    2. Delay (Atraso) = order_delivered_customer_date - order_estimated_delivery_date
    3. Tempo do despache da loja = order_delivered_carrier_date - order_approved_at

In [4]:
import pandas as pd
df_raw = pd.read_csv('dataset/raw/orders_dataset.csv')
df_processed = df_raw[df_raw['order_status'] == 'delivered'].copy()
print(df_raw.shape)
print(df_processed.shape)
df_processed.head()

(99441, 8)
(96478, 8)


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [5]:
df_processed['order_purchase_timestamp'] = pd.to_datetime(df_processed['order_purchase_timestamp'])
df_processed['order_approved_at'] = pd.to_datetime(df_processed['order_approved_at'])
df_processed['order_delivered_carrier_date'] = pd.to_datetime(df_processed['order_delivered_carrier_date'])
df_processed['order_delivered_customer_date'] = pd.to_datetime(df_processed['order_delivered_customer_date'])
df_processed['order_estimated_delivery_date'] = pd.to_datetime(df_processed['order_estimated_delivery_date'])

Criando novas colunas !!!

- tempo_entrega_real = quanto tempo demorou no total?
- tempo_delay = Se o resultado for negativo, chegou adiantado. Se positivo, atrasou. 
- tempo_despache = A loja demorou para despachar?

In [6]:
df_processed['tempo_entrega_real'] = df_processed['order_delivered_customer_date'] - df_processed['order_purchase_timestamp']
df_processed['tempo_delay'] = df_processed['order_delivered_customer_date'] - df_processed['order_estimated_delivery_date']
df_processed['tempo_despache'] = df_processed['order_delivered_carrier_date'] - df_processed['order_approved_at']
df_processed[['tempo_entrega_real', 'tempo_delay', 'tempo_despache']].head()
print(df_processed.shape)

(96478, 11)


In [7]:
colunas_finais = [
    'order_id', 
    'customer_id', 
    'order_purchase_timestamp',  # Mantendo a referência temporal
    'tempo_entrega_real', 
    'tempo_delay', 
    'tempo_despache'
]

table_of_times = df_processed[colunas_finais]

table_of_times.to_csv('dataset/processed/orders_processed.csv', index=False)

print(table_of_times.columns)
print(table_of_times.head())

Index(['order_id', 'customer_id', 'order_purchase_timestamp',
       'tempo_entrega_real', 'tempo_delay', 'tempo_despache'],
      dtype='str')
                           order_id                       customer_id  \
0  e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d   
1  53cdb2fc8bc7dce0b6741e2150273451  b0830fb4747a6c6d20dea0b8c802d7ef   
2  47770eb9100c2d0c44946d9cf07ec65d  41ce2a54c0b03bf3443c3d931a367089   
3  949d5b44dbf5de918fe9c16f97b45f8a  f88197465ea7920adcdbec7375364d82   
4  ad21c59c0840e6cb83a9ceb5573f8159  8ab97904e6daea8866dbdbc4fb7aad2c   

  order_purchase_timestamp tempo_entrega_real        tempo_delay  \
0      2017-10-02 10:56:33    8 days 10:28:40  -8 days +21:25:13   
1      2018-07-24 20:41:37   13 days 18:46:08  -6 days +15:27:45   
2      2018-08-08 08:38:49    9 days 09:27:40 -18 days +18:06:29   
3      2017-11-18 19:28:06   13 days 05:00:36 -13 days +00:28:42   
4      2018-02-13 21:18:39    2 days 20:58:23 -10 days +18:17:02   

   tempo

FAZENDO O PROCESSAMENTO DA TABELA = order_items_dataset.csv

1. Features Financeiras (A "Dor" do Pagamento)
    A sensibilidade ao preço e ao frete afeta diretamente a exigência do cliente.

    - vl_total_pedido (Soma de Price):
        Como fazer: Soma (sum) da coluna price.
        Hipótese: Quem gasta $R\$ 2.000,00$ é mais exigente do que quem gasta $R\$ 20,00$? Ou será que quem gasta pouco tolera menos erros?

    - vl_frete_total (Soma de Freight):
        Como fazer: Soma (sum) da coluna freight_value.
        Hipótese: Fretes abusivos geram insatisfação prévia.

    - ratio_frete_produto (Razão Frete/Valor):
        Como fazer: vl_frete_total / vl_total_pedido.
        Por que é crucial: Pagar $R\$ 50,00$ de frete numa geladeira de $R\$ 2.000,00$ é aceitável (2.5%). Pagar $R\$ 50,00$ de frete numa capinha de celular de $R\$ 20,00$ é revoltante (250%). Esse número captura o sentimento de "injustiça".

In [11]:
import pandas as pd

# Carregando (exemplo)
df_items = pd.read_csv('dataset/raw/order_items_dataset.csv')

# Agregação
df_agg = df_items.groupby('order_id').agg({
    'price': 'sum',                # Valor total dos produtos
    'freight_value': 'sum',        # Valor total do frete
    'order_item_id': 'count',      # Quantidade de itens
    'seller_id': 'nunique'         # Quantidade de vendedores distintos
}).reset_index()

# Renomeando colunas para ficar organizado
df_agg.columns = ['order_id', 'total_produto', 'total_frete', 'qtd_itens', 'qtd_vendedores']

# Criando a Feature de Razão (Tratando divisão por zero se houver brinde)
df_agg['ratio_frete'] = df_agg['total_frete'] / df_agg['total_produto']

# Agora df_agg tem UMA linha por pedido e está pronta para o merge com a tabela de Reviews/Orders
print(df_agg.columns)
print(df_agg.head())

Index(['order_id', 'total_produto', 'total_frete', 'qtd_itens',
       'qtd_vendedores', 'ratio_frete'],
      dtype='str')
                           order_id  total_produto  total_frete  qtd_itens  \
0  00010242fe8c5a6d1ba2dd792cb16214          58.90        13.29          1   
1  00018f77f2f0320c557190d7a144bdd3         239.90        19.93          1   
2  000229ec398224ef6ca0657da4fc703e         199.00        17.87          1   
3  00024acbcdf0a6daa1e931b038114c75          12.99        12.79          1   
4  00042b26cf59d7ce69dfabb4e55b4fd9         199.90        18.14          1   

   qtd_vendedores  ratio_frete  
0               1     0.225637  
1               1     0.083076  
2               1     0.089799  
3               1     0.984604  
4               1     0.090745  


In [12]:
# Realizando o merge (junção)
df_analise = pd.merge(
    left=table_of_times,     # Tabela principal (datas/status)
    right=df_items,          # Tabela de detalhes (preço/frete/qtd)
    on='order_id',           # A "chave" que conecta as duas
    how='inner'              # O tipo de junção
)

# Visualizando o resultado
df_analise.head()

,order_id,customer_id,order_purchase_timestamp,tempo_entrega_real,tempo_delay,tempo_despache,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,2017-10-02 10:56:33,8 days 10:28:40,-8 days +21:25:13,2 days 08:47:45,1,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,2018-07-24 20:41:37,13 days 18:46:08,-6 days +15:27:45,0 days 11:06:33,1,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,2018-07-30 03:24:27,118.70,22.76
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,2018-08-08 08:38:49,9 days 09:27:40,-18 days +18:06:29,0 days 04:54:37,1,aa4383b373c6aca5d8797843e5594415,4869f7a5dfa277a7dca6462dcf3b52b2,2018-08-13 08:55:23,159.90,19.22
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,2017-11-18 19:28:06,13 days 05:00:36,-13 days +00:28:42,3 days 17:54:00,1,d0b61bfb1de832b15ba9d266ca96e5b0,66922902710d126a0e7d26b0e3805106,2017-11-23 19:45:59,45.00,27.20
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,2018-02-13 21:18:39,2 days 20:58:23,-10 days +18:17:02,0 days 21:26:05,1,65266b2da20d04dbe00c5c2d3bb7859e,2c9e548be18521d1c43cde1c582c6de8,2018-02-19 20:31:37,19.90,8.72
